In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 
import tensorflow as tf
from scipy.io import wavfile as wav

In [2]:
audio_dataset_path='splitSound/'
metadata=pd.read_csv('data.csv')
mfcc = 60

def features_extractor(filename):
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=mfcc)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),str(row["file_name"]))
    final_class_labels=row["class_name"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

extracted_features_df = pd.DataFrame(extracted_features,columns=['feature','class'])

X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

num_labels = 7

model=Sequential()
# 1. hidden layer
model.add(Dense(125,input_shape=(mfcc,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# 2. hidden layer
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# 3. hidden layer
model.add(Dense(125))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# output layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

4632it [02:45, 28.05it/s]
2022-03-29 21:35:28.953716: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-29 21:35:28.954209: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


In [3]:
epochscount = 50
num_batch_size = 32

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=epochscount, validation_data=(X_test, y_test), verbose=1)

Epoch 1/50


2022-03-29 21:37:00.928489: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  7/116 [>.............................] - ETA: 0s - loss: 39.9701 - accuracy: 0.1562 

2022-03-29 21:37:01.285605: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


116/116 [==============================] - 2s 10ms/step - loss: 10.2650 - accuracy: 0.2629 - val_loss: 2.2193 - val_accuracy: 0.3247
Epoch 2/50
  1/116 [..............................] - ETA: 0s - loss: 4.6526 - accuracy: 0.2812

2022-03-29 21:37:02.450407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


116/116 [==============================] - 1s 9ms/step - loss: 2.7994 - accuracy: 0.4359 - val_loss: 1.5452 - val_accuracy: 0.4250
Epoch 3/50
116/116 [==============================] - 1s 9ms/step - loss: 1.8472 - accuracy: 0.5506 - val_loss: 1.4581 - val_accuracy: 0.4045
Epoch 4/50
116/116 [==============================] - 1s 9ms/step - loss: 1.3886 - accuracy: 0.6202 - val_loss: 1.2599 - val_accuracy: 0.5458
Epoch 5/50
116/116 [==============================] - 1s 9ms/step - loss: 1.0929 - accuracy: 0.6864 - val_loss: 1.0624 - val_accuracy: 0.6419
Epoch 6/50
116/116 [==============================] - 1s 9ms/step - loss: 0.8906 - accuracy: 0.7385 - val_loss: 0.9735 - val_accuracy: 0.7184
Epoch 7/50
116/116 [==============================] - 1s 9ms/step - loss: 0.8115 - accuracy: 0.7644 - val_loss: 0.8834 - val_accuracy: 0.7605
Epoch 8/50
116/116 [==============================] - 1s 10ms/step - loss: 0.7225 - accuracy: 0.7895 - val_loss: 0.8305 - val_accuracy: 0.8382
Epoch 9/50
116/1

In [4]:
validation_test_set_accuracy = model.evaluate(X_test,y_test,verbose=0)
print(validation_test_set_accuracy[1])
print(validation_test_set_accuracy[0])

0.9697951078414917
0.09370702505111694


In [5]:
result_classes = ["azerbaycan","kazakca", "uygurca","kirgizca","tatarca","turkmence","ozbekce"]
file_name = "ozbekce"
for i in range(1,11):
    filename="testSound/"+file_name+"_"+str(i)+".wav"
    sound_signal, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=mfcc)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    mfccs_scaled_features = mfccs_scaled_features.reshape(1,-1)
    result_array = model.predict(mfccs_scaled_features)
    result = np.argmax(result_array[0])
    print(result_classes[result]) 

2022-03-29 21:38:01.699699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


ozbekce
ozbekce
ozbekce
ozbekce
ozbekce
ozbekce
ozbekce
ozbekce
turkmence
turkmence
